# importing the required library

In [1]:
import re
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Extract the Date time
def date_time(s):
    pattern = r'^(\d+/\d+/\d+), (\d+:\d+\s?[APMapm]+) - '
    result = re.match(pattern, s)
    return bool(result)

# Extract contacts
def find_contact(s):
    s = s.split(":")
    return len(s) == 2

# Extract Message
def get_message(line):
    split_line = line.split(' - ')
    datetime = split_line[0]
    date, time = datetime.split(', ')
    message = " ".join(split_line[1:])
    
    if find_contact(message):
        split_message = message.split(": ")
        author = split_message[0]
        message = split_message[1]
    else:
        author = None
    return date, time, author, message

# Read chat data from file
data = []
conversation_file = 'WhatsApp Chat with Harsh.txt'
with open(conversation_file, encoding="utf-8") as fp:
    fp.readline()  # Skip the first line (metadata/header)
    message_buffer = []
    date, time, author = None, None, None
    for line in fp:
        line = line.strip()
        if date_time(line):
            if message_buffer:
                data.append([date, time, author, " ".join(message_buffer)])
            message_buffer.clear()
            date, time, author, message = get_message(line)
            message_buffer.append(message)
        else:
            message_buffer.append(line)

# Create a DataFrame
df = pd.DataFrame(data, columns=["Date", "Time", "Author", "Message"])
df['Date'] = pd.to_datetime(df['Date'])

# Initialize the VADER sentiment analyzer
sentiments = SentimentIntensityAnalyzer()

# Calculate sentiment scores
df["Positive"] = [sentiments.polarity_scores(msg)["pos"] for msg in df["Message"]]
df["Negative"] = [sentiments.polarity_scores(msg)["neg"] for msg in df["Message"]]
df["Neutral"] = [sentiments.polarity_scores(msg)["neu"] for msg in df["Message"]]

# Display the first 10 rows
df.head(5)


C:\Users\DELL\AppData\Local\Temp\ipykernel_13748\3100879398.py:51: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'])


,Date,Time,Author,Message,Positive,Negative,Neutral
0,2023-01-07,9:27 pm,Harsh,Dare kya tha,0.0,0.0,1.0
1,2023-01-07,9:39 pm,Røhît Kümãr,Bss apna pic dalna tha yr mention krke,0.0,0.0,1.0
2,2023-01-07,9:39 pm,Røhît Kümãr,And song add krna tha,0.0,0.0,1.0
3,2023-06-07,6:06 pm,Røhît Kümãr,😂,0.0,0.0,0.0
4,2023-06-07,6:07 pm,Røhît Kümãr,Bhj toh acha wla,0.0,0.0,1.0


In [2]:
x=sum(df["Positive"])
y=sum(df["Negative"])
z=sum(df["Neutral"])


def score(a,b,c):
    if (a>b) and (a>c):
        print("it's a positive sentiment")
    if (b>a) and (b>c):
        print("it's a negative sentiment")
    if (c>a) and (c>b):
        print("it's a neutral one")

score(x,y,z)

it's a neutral one
